In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing


2023-01-04 13:28:10.846238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 13:28:10.962367: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-04 13:28:10.967708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 13:28:10.967725: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [3]:
latent_dim = 3
inputlen = 27
units = 514

# Encoder
encoder_inputs = keras.Input(shape=(inputlen,))
x = layers.Dense(units=units, activation='relu')(encoder_inputs)
x = layers.Dense(units=units//2, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(units=units//2, activation='relu')(latent_inputs)
x = layers.Dense(units=units, activation='relu')(x)
decoder_outputs = layers.Dense(inputlen, activation='linear')(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 27)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 514)          14392       ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 257)          132355      ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 3)            774         ['dense_1[0][0]']                
                                                                                            

2023-01-04 13:28:12.607704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-04 13:28:12.607719: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-04 13:28:12.607731: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmhpc): /proc/driver/nvidia/version does not exist
2023-01-04 13:28:12.607876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #print("reconstruction = " + str(reconstruction))
            #print("z_mean = " + str(z_mean)+"\n")
            #print("z_log_var = " + str(z_log_var)+"\n")
            #print("z = " + str(z)+"\n")
            
            reconstruction_loss = keras.losses.mean_squared_error(data, reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            #print("First kl_loss = "+str(kl_loss)+"\n")
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss,axis=1)) 
            #mprint("Second kl_loss = "+str(kl_loss)+"\n")
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [5]:
df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/EnvDataAutoencoder/Data/Imputed_Metadata_metaGmetaT_by_OceanLayer.csv",sep=",")
selectcols = ["Latitude","Longitude","Depth.nominal","Temperature","Oxygen","ChlorophyllA","Carbon.total","Salinity","Gradient.Surface.temp","Fluorescence","CO3","HCO3","Density","PO4","NO3","Si","Alkalinity.total","Ammonium.5m","Lyapunov","NO2","Depth.Min.O2","NO2NO3","Nitracline","Brunt.Vaisala","Iron.5m","Depth.Max.O2","Okubo.Weiss"]
df = df[selectcols]
print(df.shape)
df = preprocessing.RobustScaler().fit_transform(df)



(228, 27)


In [6]:
#checkpoint_filepath = './model_chkpnt'
#chkpt = tf.keras.callbacks.ModelCheckpoint(
#    filepath=checkpoint_filepath,
#    save_best_only=True,
#    monitor='loss',
#    verbose=1)

tb = keras.callbacks.TensorBoard(log_dir="./vae_tb",update_freq="epoch")
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1000)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(df, epochs=5000, batch_size=128, callbacks=[tb,earlystop] )
vae.save_weights('./vae_checkpoint/my_checkpoint')

Epoch 1/5000
2/2 [==============================] - 1s 17ms/step - loss: 14.4277 - reconstruction_loss: 16.3332 - kl_loss: 0.8942
Epoch 2/5000
2/2 [==============================] - 0s 7ms/step - loss: 17.9022 - reconstruction_loss: 16.1394 - kl_loss: 0.7324
Epoch 3/5000
2/2 [==============================] - 0s 6ms/step - loss: 18.0625 - reconstruction_loss: 15.5840 - kl_loss: 0.8686
Epoch 4/5000
2/2 [==============================] - 0s 7ms/step - loss: 18.4248 - reconstruction_loss: 15.0712 - kl_loss: 1.2030
Epoch 5/5000
2/2 [==============================] - 0s 7ms/step - loss: 12.6996 - reconstruction_loss: 15.1912 - kl_loss: 0.6775
Epoch 6/5000
2/2 [==============================] - 0s 6ms/step - loss: 15.5036 - reconstruction_loss: 14.6113 - kl_loss: 1.0112
Epoch 7/5000
2/2 [==============================] - 0s 6ms/step - loss: 13.1709 - reconstruction_loss: 13.0258 - kl_loss: 2.1646
Epoch 8/5000
2/2 [==============================] - 0s 7ms/step - loss: 13.0504 - reconstruction